# Introduction to PL/pgSQL CASE Statment

The `case` statement selects a `when` section to execute from a list of `when` sections based on a condition.

The `case` statement has two forms:
* Simple `case` statement
* Searched `case` statement

Notice that you should not be confused about the `case` statement and `case expression`. 

The `case` expression evaluates to a value while the `case` statement selects a section to execute based on conditions.

# 1) Simple case statement

```postgresql
case search-expression
   when expression_1 [, expression_2, ...] then
      when-statements
  [ ... ]
  [else
      else-statements ]
END case; 
```

* The `search-expression` is an expression that evaluates to a result.
* The `case` statement compares the result of the `search-expression` with the `expression` in each `when` branch using the equal operator (`=`) from top to bottom.
* If the `case` statement finds a match, it will execute the corresponding `when` section. Additionally, it stops checking the remaining `when` sections
* If the `case` statement cannot find any match, it will execute the `else` section.
* The `else` section is optional. If the result of the `search-expression` does not match the `expression` in the `when` sections and the `else` section does not exist, the `case` statement will raise a `case_not_found` exception.

**Example**:

```postgresql
do $$
declare
	rate film.rental_rate%type;
	price_segment varchar(50);
begin
    -- get the rental rate
    select rental_rate into rate
    from film
    where film_id = 100;

	-- assign the price segment
	if found then
		case rate
		   when 0.99 then
              price_segment =  'Mass';
		   when 2.99 then
              price_segment = 'Mainstream';
		   when 4.99 then
              price_segment = 'High End';
		   else
	    	  price_segment = 'Unspecified';
		   end case;

		raise notice '%', price_segment;
	else
		raise notice 'film not found';
    end if;
end; $$ 
```

**Output:**

```
NOTICE:  High End
```

# 2) Searched case statement

```postgresql
case
    when boolean-expression-1 then
      statements
  [ when boolean-expression-2 then
      statements
    ... ]
  [ else
      statements ]
end case;
```

* In this syntax, the `case` statement evaluates the boolean expressions sequentially from top to bottom until it finds an expression that evaluates to `true`.
* Subsequently, the `case` statement executes the corresponding `when` section and immediately stops searching for the remaining expressions.
* If no expression evaluates to `true`, the `case` statement will execute the `else` section.
* The `else` section is optional. If you omit the `else` section and there is no expression evaluated to `true`, the `case` statement will raise the `case_not_found` exception.

**Example:**

```postgresql
do $$
declare
    total_payment numeric;
    service_level varchar(25) ;
begin
     select sum(amount) into total_payment
     from Payment
     where customer_id = 100;

	 if found then
	    case
		   when total_payment > 200 then
               service_level = 'Platinum' ;
           when total_payment > 100 then
	           service_level = 'Gold' ;
           else
               service_level = 'Silver' ;
        end case;
		raise notice 'Service Level: %', service_level;
     else
	    raise notice 'Customer not found';
	 end if;
end; $$ 
```

